# Analysis for Differential Jet Mass in Z+Jets events with NanoAODv9

In [ ]:
import awkward as ak
import numpy as np
import time
import coffea
import uproot
import hist
import vector
print("awkward version ", ak.__version__)
print("coffea version ", coffea.__version__)
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from collections import defaultdict
import pickle
from distributed.diagnostics.plugin import UploadDirectory
import os

### Automatically reload any modules used

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from smp_utils import *
from response_maker_nanov9_lib import *
from response_maker_nanov9 import *

In [ ]:
from dask.distributed import Client

client = Client("tls://rappoccio-40gmail-2ecom.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client

In [ ]:
client.register_worker_plugin(UploadDirectory("/home/cms-jovyan/QJetMassUproot", restart=True, update_path=True), nanny=True)
print(client.run(os.listdir, "dask-worker-space") )

### If desired, uncomment these to move or remove old pkl files

In [ ]:
#! mv qjetmass_zjets_gen.pkl qjetmass_zjets_gen_old.pkl
#! mv qjetmass_zjets_reco.pkl qjetmass_zjets_reco_old.pkl

### If you want to rerun, here is the line. Otherwise keep it commented to just read the pkl file. 

In [ ]:
#response_maker_nanov9(testing=False, do_gen=False, client=None, prependstr="/mnt/data/cms")
response_maker_nanov9(testing=False, do_gen=True, client=client)

In [ ]:
response_maker_nanov9(testing=False, do_gen=False, client=client)
#response_maker_nanov9(testing=True, do_gen=True, client=None, prependstr="/mnt/data/cms")

In [ ]:
#response_maker_nanov9(testing=True, do_gen=False, client=None, prependstr="/mnt/data/cms")

### Open pkl file for the output histograms

In [ ]:
with open("qjetmass_zjets_gen.pkl", "rb") as f:
    output = pickle.load( f )

In [ ]:
output.keys()

In [ ]:
output["cutflow"]